In [1]:
import pandas as pd
import numpy as np
from sql_functions import *
from normal_round import normal_round as normal_round

In [2]:
pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)

In [3]:
df_bund_abs = get_dataframe('SELECT * FROM public.bund_jugend_tat_absolut_2022_until_2018;')
df_bund_rel = get_dataframe('SELECT * FROM public.bund_jugend_tat_relativ_2022_until_2018;')

In [4]:
df_bund_abs.head(5)

,schluessel,straftat,sexus,tatverdaechtige_insgesamt,kinder_10_bis_unter_12,kinder_12_bis_unter_14,kinder_10_bis_unter_14,kinder_unter_14,jugendliche_14_bis_unter_16,jugendliche_16_bis_unter_18,jugendliche_14<18,heranwachsende_18_bis_unter_21,jugendl_u_heranwachsende_14_bis_unter_21,erwachsene_ab_21,year
0,------,Straftaten insgesamt,M,1541130,11882,25507,37389,48288,52244,76029,128273,147013,275286,1217556,2018
1,------,Straftaten insgesamt,W,510136,4377,12991,17368,22315,24451,24707,49158,38510,87668,400153,2018
2,------,Straftaten insgesamt,X,2051266,16259,38498,54757,70603,76695,100736,177431,185523,362954,1617709,2018
3,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,M,43036,310,1168,1478,1710,2861,2789,5650,4018,9668,31658,2018
4,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,W,2500,76,309,385,408,339,133,472,133,605,1487,2018


In [5]:
#df_bund_rel.tail()

### Calculate most occuring crime per state - rank and percentage of total

In [6]:
df_1 = df_bund_abs.copy()
df_1 = df_1[['schluessel', 'straftat', 'sexus', 'year', 'jugendliche_14_bis_unter_16']]
df_1 = df_1.rename(columns={'jugendliche_14_bis_unter_16': 'offenders'})

df_2 = df_1[df_1['schluessel'] == '------']
df_2 = df_2[['sexus', 'year', 'offenders']]
df_bund_straftaten_insg_absolut = df_2.copy() # needed later

df_3 = df_1[df_1['schluessel'] != '------']

df_3['rank'] = (
    df_3.groupby(['sexus', 'year'])['offenders']
        .rank(ascending=False, method='first')
        .astype(int)
)

df_4 = df_3.merge(df_2, on=['sexus', 'year'], how='left')

df_4['certain_crime_percent_of_total_crime'] = df_4['offenders_x'] / df_4['offenders_y'] * 100
df_4 = df_4.drop(columns='offenders_y')
df_4 = df_4.reset_index()
df_4 = df_4.drop(columns='index')

values = df_4['certain_crime_percent_of_total_crime'].tolist()
new_values = []
for v in values:
    new_v = normal_round(num=v, ndigits=2)
    new_values.append(new_v)
    
df_4['certain_crime_percent_of_total_crime'] = new_values

df_4 = df_4[['schluessel', 'sexus', 'year', 'straftat', 'rank', 'certain_crime_percent_of_total_crime']]

crime_rank_and_percent_of_total = df_4.copy()
crime_rank_and_percent_of_total.head()

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18874/2239140198.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3['rank'] = (


,schluessel,sexus,year,straftat,rank,certain_crime_percent_of_total_crime
0,100000,M,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,5.48
1,100000,W,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,1.39
2,100000,X,2018,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5,4.17
3,210000,M,2018,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",6,3.63
4,210000,W,2018,"Raub, räuberische Erpressung und räuberischer Angriff auf Kraftfahrer §§ 249-252, 255, 316a StGB",7,0.92


### Create overview - number of offenders per state - absolut and relativ

In [7]:
df_10 = df_bund_rel.copy()
df_10 = df_10.rename(columns={'jugendliche_14_bis_unter_16_rel': 'offenders_rel'})
df_10 = df_10[(df_10['schluessel'] == '------')]

jugendkrim_insg_rel = df_10['offenders_rel'].tolist()
jugendkrim_insg_rel_rounded = []
for v in jugendkrim_insg_rel:
    v_new = int(normal_round(v, 0))
    jugendkrim_insg_rel_rounded.append(v_new)
df_10['offenders_rel'] = jugendkrim_insg_rel_rounded

df_10 = df_10[['sexus', 'year', 'offenders_rel']]
df_10 = df_10.reset_index()
df_10 = df_10.drop(columns='index')

df_11 = df_bund_straftaten_insg_absolut.merge(df_10, on=['sexus', 'year'], how='left')
df_bund_straftaten_insg_abs_u_rel = df_11.copy()
df_bund_straftaten_insg_abs_u_rel.head()

,sexus,year,offenders,offenders_rel
0,M,2018,52244,6723
1,W,2018,24451,3347
2,X,2018,76695,5088
3,M,2019,54636,7059
4,W,2019,25226,3456


### Merge crimes that <br>rank on 1, 2, and 3 <br>and <br>overall number of crimes - absolut and relativ <br>to every state for every year (->one DataFrame)

In [8]:
df_20 = crime_rank_and_percent_of_total.copy()
df_rank_1 = df_20[df_20['rank'] == 1]
df_rank_2 = df_20[df_20['rank'] == 2]
df_rank_3 = df_20[df_20['rank'] == 3]
df_rank_4 = df_20[df_20['rank'] == 4]
df_rank_5 = df_20[df_20['rank'] == 5]
df_rank_1.head()

,schluessel,sexus,year,straftat,rank,certain_crime_percent_of_total_crime
6,220000,M,2018,"Körperverletzung §§ 223-227, 229, 231 StGB",1,24.79
19,*26*00,W,2018,Ladendiebstahl insgesamt,1,42.99
20,*26*00,X,2018,Ladendiebstahl insgesamt,1,29.59
36,220000,M,2019,"Körperverletzung §§ 223-227, 229, 231 StGB",1,25.39
49,*26*00,W,2019,Ladendiebstahl insgesamt,1,41.98


In [9]:
df_30 = df_bund_straftaten_insg_abs_u_rel.copy()

df_31 = df_30.merge(df_rank_1, on=['sexus', 'year'], how='left')
df_31 = df_31.rename(columns={'straftat': 'crimes_on_rank_1',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_1_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_1'})
df_31 = df_31.drop(columns='rank')

df_32 = df_31.merge(df_rank_2, on=['sexus', 'year'], how='left')
df_32 = df_32.rename(columns={'straftat': 'crimes_on_rank_2',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_2_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_2'})
df_32 = df_32.drop(columns='rank')

df_33 = df_32.merge(df_rank_3, on=['sexus', 'year'], how='left')
df_33 = df_33.rename(columns={'straftat': 'crimes_on_rank_3',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_3_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_3'})
df_33 = df_33.drop(columns='rank')

df_34 = df_33.merge(df_rank_4, on=['sexus', 'year'], how='left')
df_34 = df_34.rename(columns={'straftat': 'crimes_on_rank_4',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_4_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_4'})
df_34 = df_34.drop(columns='rank')

df_35 = df_34.merge(df_rank_5, on=['sexus', 'year'], how='left')
df_35 = df_35.rename(columns={'straftat': 'crimes_on_rank_5',
                               'certain_crime_percent_of_total_crime': 'percentage_of_rank_5_on_crime_total',
                               'schluessel': 'schluessel_crimes_on_rank_5'})
df_35 = df_35.drop(columns='rank')

df_bund_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5 = df_35.copy() 
df_bund_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5.head()

,sexus,year,offenders,offenders_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
0,M,2018,52244,6723,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",24.79,*26*00,Ladendiebstahl insgesamt,23.32,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),16.15,674000,Sachbeschädigung §§ 303-305a StGB,15.03,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5.48
1,W,2018,24451,3347,*26*00,Ladendiebstahl insgesamt,42.99,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",17.03,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),10.91,674000,Sachbeschädigung §§ 303-305a StGB,6.07,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,1.39
2,X,2018,76695,5088,*26*00,Ladendiebstahl insgesamt,29.59,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.31,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),14.48,674000,Sachbeschädigung §§ 303-305a StGB,12.18,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,4.17
3,M,2019,54636,7059,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",25.39,*26*00,Ladendiebstahl insgesamt,22.23,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),15.39,674000,Sachbeschädigung §§ 303-305a StGB,14.77,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,8.16
4,W,2019,25226,3456,*26*00,Ladendiebstahl insgesamt,41.98,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",18.39,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),10.78,674000,Sachbeschädigung §§ 303-305a StGB,5.76,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,2.63


### Calculate Growth Rate

In [10]:
df_51 = df_bund_abs.copy()
df_51 = df_51[["schluessel", "straftat", "year", 'sexus', "jugendliche_14_bis_unter_16"]]
df_51 = df_51[df_51["schluessel"] == "------"]
df_51 = df_51[["year", 'sexus', "jugendliche_14_bis_unter_16"]]

df_53 = pd.DataFrame(columns = ["year", 'sexus', "jugendliche_14_bis_unter_16", "growth_rate_abs"])

gender = ['X', 'W', 'M']

for g in gender:
    df_52 = df_51[df_51["sexus"] == g]
    df_52["growth_rate_abs"] = df_52['jugendliche_14_bis_unter_16'].pct_change() * 100
    df_53 = pd.concat([df_53, df_52])

df_53 = df_53.drop(columns='jugendliche_14_bis_unter_16')

df_bund_growth_rate_abs = df_53.copy()
df_bund_growth_rate_abs.head(5)

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18874/3429287429.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_52["growth_rate_abs"] = df_52['jugendliche_14_bis_unter_16'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18874/3429287429.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_52["growth_rate_abs"] = df_52['jugendliche_14_bis_unter_16'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18874/3429287429.py:12: Setti

,year,sexus,growth_rate_abs
2,2018,X,NaN
35,2019,X,4.129344
68,2020,X,-9.571511
101,2021,X,-3.806530
134,2022,X,31.118916


In [11]:
df_61 = df_bund_rel.copy()
df_61 = df_61[["schluessel", "straftat", "year", 'sexus', "jugendliche_14_bis_unter_16_rel"]]
df_61 = df_61[df_61["schluessel"] == "------"]
df_61 = df_61[["year", 'sexus', "jugendliche_14_bis_unter_16_rel"]]

df_63 = pd.DataFrame(columns = ["year", 'sexus', "jugendliche_14_bis_unter_16_rel", "growth_rate_rel"])

gender = ['X', 'W', 'M']

for g in gender:
    df_62 = df_61[df_61["sexus"] == g]
    df_62["growth_rate_rel"] = df_62['jugendliche_14_bis_unter_16_rel'].pct_change() * 100
    df_63 = pd.concat([df_63, df_62])

df_63 = df_63.drop(columns='jugendliche_14_bis_unter_16_rel')

df_bund_growth_rate_rel = df_63.copy()
df_bund_growth_rate_rel.head(5)

/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18874/1612305001.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_62["growth_rate_rel"] = df_62['jugendliche_14_bis_unter_16_rel'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18874/1612305001.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_62["growth_rate_rel"] = df_62['jugendliche_14_bis_unter_16_rel'].pct_change() * 100
/var/folders/1s/wcz2lmzj3ql58wp26sv0rw3h0000gn/T/ipykernel_18874/1612305001.py:1

,year,sexus,growth_rate_rel
2,2018,X,NaN
35,2019,X,4.371185
68,2020,X,-9.170258
101,2021,X,-2.639962
134,2022,X,29.759561


In [12]:
df_growth_rate_14_to_under_16 = df_bund_growth_rate_abs.merge(df_bund_growth_rate_rel, on=['year', 'sexus'], how='left')
df_growth_rate_14_to_under_16.head()

,year,sexus,growth_rate_abs,growth_rate_rel
0,2018,X,NaN,NaN
1,2019,X,4.129344,4.371185
2,2020,X,-9.571511,-9.170258
3,2021,X,-3.806530,-2.639962
4,2022,X,31.118916,29.759561


In [13]:
values_growth_rate_abs = df_growth_rate_14_to_under_16["growth_rate_abs"].tolist()
new_values_abs = []
for v in values_growth_rate_abs:
    if np.isnan(v):
        new_v = v
    else: 
        new_v = normal_round(v, 2)
    new_values_abs.append(new_v)
df_growth_rate_14_to_under_16["growth_rate_abs"] = new_values_abs

values_growth_rate_rel = df_growth_rate_14_to_under_16["growth_rate_rel"].tolist()
new_values_rel = []
for v in values_growth_rate_rel:
    if np.isnan(v):
        new_v = v
    else: 
        new_v = normal_round(v, 2)
    new_values_rel.append(new_v)
df_growth_rate_14_to_under_16["growth_rate_rel"] = new_values_rel

In [14]:
df_growth_rate_14_to_under_16.head()

,year,sexus,growth_rate_abs,growth_rate_rel
0,2018,X,NaN,NaN
1,2019,X,4.13,4.37
2,2020,X,-9.56,-9.16
3,2021,X,-3.80,-2.63
4,2022,X,31.12,29.76


In [15]:
df_overview_state_14_to_under_16 = df_bund_straftaten_insg_abs_u_rel_ranks_1_2_3_4_5.merge(df_growth_rate_14_to_under_16, on=['sexus', 'year'], how='left')

df_overview_state_14_to_under_16['age_group'] = 'jugendliche_14_bis_unter_16'

# bring in correct order
df_overview_state_14_to_under_16 = df_overview_state_14_to_under_16[['year', 'sexus', 'age_group', 'offenders', 'offenders_rel',
       'growth_rate_abs', 'growth_rate_rel',
       'schluessel_crimes_on_rank_1', 'crimes_on_rank_1',
       'percentage_of_rank_1_on_crime_total', 'schluessel_crimes_on_rank_2',
       'crimes_on_rank_2', 'percentage_of_rank_2_on_crime_total',
       'schluessel_crimes_on_rank_3', 'crimes_on_rank_3',
       'percentage_of_rank_3_on_crime_total', 'schluessel_crimes_on_rank_4',
       'crimes_on_rank_4', 'percentage_of_rank_4_on_crime_total',
       'schluessel_crimes_on_rank_5', 'crimes_on_rank_5',
       'percentage_of_rank_5_on_crime_total']]

df_overview_state_14_to_under_16.tail(5)

,year,sexus,age_group,offenders,offenders_rel,growth_rate_abs,growth_rate_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
10,2021,W,jugendliche_14_bis_unter_16,21191,2954,-3.99,-2.71,*26*00,Ladendiebstahl insgesamt,37.86,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",17.01,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),9.97,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,6.84,674000,Sachbeschädigung §§ 303-305a StGB,5.88
11,2021,X,jugendliche_14_bis_unter_16,69469,4696,-3.80,-2.63,*26*00,Ladendiebstahl insgesamt,22.69,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",20.52,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,13.20,674000,Sachbeschädigung §§ 303-305a StGB,12.30,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),12.15
12,2022,M,jugendliche_14_bis_unter_16,61843,8025,28.10,26.65,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",24.90,*26*00,Ladendiebstahl insgesamt,21.79,674000,Sachbeschädigung §§ 303-305a StGB,14.14,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,13.39,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),9.88
13,2022,W,jugendliche_14_bis_unter_16,29244,4038,38.00,36.71,*26*00,Ladendiebstahl insgesamt,49.11,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",16.92,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),7.45,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5.10,674000,Sachbeschädigung §§ 303-305a StGB,5.04
14,2022,X,jugendliche_14_bis_unter_16,91087,6093,31.12,29.76,*26*00,Ladendiebstahl insgesamt,30.56,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.34,674000,Sachbeschädigung §§ 303-305a StGB,11.22,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,10.73,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),9.10


In [16]:
df_overview_state_14_to_under_16 = df_overview_state_14_to_under_16.fillna('n.a.')

In [17]:
df_overview_state_14_to_under_16.head()

,year,sexus,age_group,offenders,offenders_rel,growth_rate_abs,growth_rate_rel,schluessel_crimes_on_rank_1,crimes_on_rank_1,percentage_of_rank_1_on_crime_total,schluessel_crimes_on_rank_2,crimes_on_rank_2,percentage_of_rank_2_on_crime_total,schluessel_crimes_on_rank_3,crimes_on_rank_3,percentage_of_rank_3_on_crime_total,schluessel_crimes_on_rank_4,crimes_on_rank_4,percentage_of_rank_4_on_crime_total,schluessel_crimes_on_rank_5,crimes_on_rank_5,percentage_of_rank_5_on_crime_total
0,2018,M,jugendliche_14_bis_unter_16,52244,6723,n.a.,n.a.,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",24.79,*26*00,Ladendiebstahl insgesamt,23.32,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),16.15,674000,Sachbeschädigung §§ 303-305a StGB,15.03,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,5.48
1,2018,W,jugendliche_14_bis_unter_16,24451,3347,n.a.,n.a.,*26*00,Ladendiebstahl insgesamt,42.99,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",17.03,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),10.91,674000,Sachbeschädigung §§ 303-305a StGB,6.07,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,1.39
2,2018,X,jugendliche_14_bis_unter_16,76695,5088,n.a.,n.a.,*26*00,Ladendiebstahl insgesamt,29.59,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",22.31,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),14.48,674000,Sachbeschädigung §§ 303-305a StGB,12.18,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,4.17
3,2019,M,jugendliche_14_bis_unter_16,54636,7059,4.58,4.99,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",25.39,*26*00,Ladendiebstahl insgesamt,22.23,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),15.39,674000,Sachbeschädigung §§ 303-305a StGB,14.77,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,8.16
4,2019,W,jugendliche_14_bis_unter_16,25226,3456,3.17,3.23,*26*00,Ladendiebstahl insgesamt,41.98,220000,"Körperverletzung §§ 223-227, 229, 231 StGB",18.39,730000,Rauschgiftdelikte (soweit nicht bereits mit anderer Schlüsselzahl erfasst),10.78,674000,Sachbeschädigung §§ 303-305a StGB,5.76,100000,Straftaten gegen die sexuelle Selbstbestimmung insgesamt,2.63


# Now Create same DataFrame for other age groups and then concat them to df_overview_fed_states

In [18]:
%store df_overview_state_14_to_under_16

Stored 'df_overview_state_14_to_under_16' (DataFrame)
